In [0]:
#Installer des modules qu'on aurait besoin
!pip install pycountry
!pip install pays

     |████████████████████████████████| 10.0MB 553kB/s 
  Created wheel for pycountry: filename=pycountry-19.8.18-py2.py3-none-any.whl size=10627361 sha256=f9b9f69f810a5fdd24b8cc1a54bb139aa2afc62008b52bc167cddcad26ef53c5
  Stored in directory: /root/.cache/pip/wheels/a2/98/bf/f0fa1c6bf8cf2cbdb750d583f84be51c2cd8272460b8b36bd3
Successfully built pycountry
     |████████████████████████████████| 112kB 2.7MB/s 
  Created wheel for pays: filename=pays-0.2.3-cp36-none-any.whl size=116955 sha256=ccdcb926dc28ea2de6907f526fca3d0ef4a7f454f07c1776fcc229b43496d520
  Stored in directory: /root/.cache/pip/wheels/00/6b/52/3741cf41083453c269d6948d6716d84363e464fb5698ee5c6f
Successfully built pays


In [0]:
import pycountry as py
import pandas as pd
import re
import numpy as np
import nltk 
from nltk import PorterStemmer, download
from nltk.corpus import stopwords
from string import punctuation
download('stopwords')
import collections as c

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_collection=pd.read_csv('/content/drive/My Drive/Colab Notebooks/collection_v51.csv')
df_article=pd.read_csv('/content/drive/My Drive/Colab Notebooks/documents_v51.csv')
df_auteur=pd.read_csv('/content/drive/My Drive/Colab Notebooks/personne_v51.csv')
df_redige=pd.read_csv('/content/drive/My Drive/Colab Notebooks/redige_v51.csv')

# Collection

In [0]:
df_collection.head()

,issn,nom_clt,nb_numeros,nb_document,pre_année_pub,der_année_pub,discpline,editeur
0,2262-3353,Hommes & Migrations,180.0,4523.0,1987.0,2008.0,"démographie, science politique",Musée de l'Histoire de l'Immigration
1,1950-6694,Revue économique,320.0,6776.0,1950.0,2002.0,économie,SciencesPo
2,1783-1768,Revue Philosophique de Louvain,441.0,12292.0,1894.0,2015.0,philosophie,Peeters
3,1957-7966,Population,320.0,12794.0,1946.0,2004.0,démographie,INED (Institut national d’études démographiques)
4,1777-5418,Revue Européenne des Migrations Internationales,49.0,814.0,1985.0,2001.0,démographie,MSH Poitiers


In [0]:
df_collection[df_collection['discpline'].isnull()]

,issn,nom_clt,nb_numeros,nb_document,pre_année_pub,der_année_pub,discpline,editeur


In [0]:
#Supprimer des collections non referencées
df_collection.drop([26,166,215], inplace=True)

In [0]:
df_collection=df_collection.fillna('')

In [0]:
df_collection['nb_numeros']=df_collection['nb_numeros'].astype(int)
df_collection['nb_document']=df_collection['nb_document'].astype(int)
df_collection['pre_année_pub']=df_collection['pre_année_pub'].astype(int)
df_collection['der_année_pub']=df_collection['der_année_pub'].astype(int)
df_collection.head()

,issn,nom_clt,nb_numeros,nb_document,pre_année_pub,der_année_pub,discpline,editeur
0,2262-3353,Hommes & Migrations,180,4523,1987,2008,"démographie, science politique",Musée de l'Histoire de l'Immigration
1,1950-6694,Revue économique,320,6776,1950,2002,économie,SciencesPo
2,1783-1768,Revue Philosophique de Louvain,441,12292,1894,2015,philosophie,Peeters
3,1957-7966,Population,320,12794,1946,2004,démographie,INED (Institut national d’études démographiques)
4,1777-5418,Revue Européenne des Migrations Internationales,49,814,1985,2001,démographie,MSH Poitiers


In [0]:
df_collection.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_collection.csv',index=False)

# Article


In [0]:
df_article[['num_doc','annee_publication']]=df_article[['num_doc','annee_publication']].astype(int)
df_article.head()

,num_doc,title,type,annee_publication,num_publication,abstract,issn
0,1,La fuite des cerveaux : le cas de l'ex-URSS,article,1992,1155,La fuite des cerveaux : le cas de l'ex-URSS......,2262-3353
1,2,"Croissance, redistribution et inégalités dans ...",article,1999,50-3,"Croissance, redistribution et inégalités dans ...",1950-6694
2,3,Que nous devons-nous les uns aux autres ? Le c...,article,2008,106-4,La fuite des cerveaux est-elle un phénomène in...,1783-1768
3,4,Chorafas Dimitri N. — La fuite des cerveaux. L...,compte-rendu,1970,25-5,Chorafas Dimitri N. — La fuite des cerveaux. L...,1957-7966
4,5,Au-delà de la « fuite des cerveaux » : la mobi...,article,1997,13-1,Au-delà de la « fuite des cerveaux » : la mobi...,1777-5418


In [0]:
df_=df_article[df_article['issn'].notnull()]

In [0]:
t_article=df_[['num_doc','title','annee_publication','issn']]
abstract=df_article[['num_doc','abstract']]

In [0]:
t_article.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_article.csv',index=False)

In [0]:
df_null=df_article[df_article['issn'].isnull()]
df_null.head()

,num_doc,title,type,annee_publication,num_publication,abstract,issn
39,40,"I. Formation, emploi, territoires (Professionn...",dossier thématique,2004,81-2,la fuite et l'attraction des cerveaux Le dével...,NaN
40,41,"La géographie, une discipline qui stimule les ...",autre,1998,22,"La géographie, une discipline qui stimule les ...",NaN
42,43,Fuite et nostalgie romantiques de la Grèce,article,1994,4,Fuite et nostalgie romantiques de la Grèce...F...,NaN
70,71,La Révolution française et l'Événement : la fu...,article,1992,104-1,"Mona Ozouf, La Révolution française et l'Événe...",NaN
84,85,L’affaire du cerveau. Quelques remarques sur l...,article,2006,25,La place et le rôle du cerveau dans la médecin...,NaN


In [0]:
for i in df_null.index:
  if i in df_collection.index:
    df_null.loc[i,['issn']]=df_collection.loc[i,['issn']][0]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
df_null[df_null['issn'].isnull()]

,num_doc,title,type,annee_publication,num_publication,abstract,issn
234,235,Le déracinement du vrai et la « fuite des dieu...,article,1994,74-2,Le déracinement du vrai et la « fuite des dieu...,NaN
250,251,Deux Universités : Strasbourg et Nancy,article,1896,32,"de Nancy, pour le nombre et la valeur du perso...",NaN
273,274,Commémoration d’un jeune poète : Charles Argentin,article,1968,211,", le délire et la foi. Insatiable dans son dés...",NaN
280,281,Adresses du directoire du département de la Lo...,travail de l'Assemblée et productions du roi e...,1791,27,Adresses du directoire du département de la Lo...,NaN
281,282,Discussion sur la délibération du directoire d...,travail de l'Assemblée et productions du roi e...,1791,27,Discussion sur la délibération du directoire d...,NaN
...,...,...,...,...,...,...,...
5009,5010,Environmental Liabilities and Régulation in Eu...,compte-rendu,1993,1,Environmental Liabilities and Régulation in Eu...,NaN
5065,5066,Histoire des relations entre l'Europe et l'Ori...,autre,1985,2,Histoire des relations entre l'Europe et l'Ori...,NaN
5066,5067,Histoire des relations entre l'Europe et l'Ori...,autre,1987,3,Histoire des relations entre l'Europe et l'Ori...,NaN
5067,5068,Histoire des relations entre l'Europe et l'Ori...,autre,1994,4,Histoire des relations entre l'Europe et l'Ori...,NaN


# Mot article

In [0]:
abstract.head()

,num_doc,abstract
0,1,La fuite des cerveaux : le cas de l'ex-URSS......
1,2,"Croissance, redistribution et inégalités dans ..."
2,3,La fuite des cerveaux est-elle un phénomène in...
3,4,Chorafas Dimitri N. — La fuite des cerveaux. L...
4,5,Au-delà de la « fuite des cerveaux » : la mobi...


In [0]:
def nettoyer_abstract(df):
       
    #Recuperer les stopswords 
    sr = nltk.corpus.stopwords.words('french')
    sc = ['cet','cette','celui','celle','celles','leurs','quoi','quel','lequel','laquelle','lesquels','lesquelles']
    stopwordss=list(set().union(sr,sc))
  
    
    #Recuperer les ponctuations
    poncts=punctuation
    poncts+='’'
    
    #lemmatisation
    stemmer=nltk.stem.snowball.FrenchStemmer()
    
    for i in df.index:
        doc=df.loc[i,['abstract']][0]
        clean_tokens=[]
        doc=str(doc)
        
        #Enlever les ponctuations
        for p in poncts :
            doc=doc.replace(p," ")
            
        doc=doc.lower()
        tokens=doc.split()

        #Enlever les stopwords et raciniser
        clean_tokens=[stemmer.stem(token) for token in tokens if (token not in stopwordss and len(token)>3)]

        #Enlever les chiffres
        clean_tokenss=[token for token in clean_tokens if token.isnumeric()==False]
        
        df.loc[i,['abstract']]=' '.join(clean_tokenss)
    
    return df

In [0]:
abstract=nettoyer_abstract(abstract)
abstract.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,num_doc,abstract
0,1,fuit cerveau urss exod inte¬ rieur cess accéle...
1,2,croissanc redistribu inégal model fuit cerveau...
2,3,fuit cerveau phénomen individuel légitim socia...
3,4,choraf dimitr fuit cerveau conquêt sav choraf ...
4,5,delà fuit cerveau mobil person haut qualifi ou...


In [0]:
mots=(' '.join(abstract['abstract'].values.tolist())).split()
mots[:5]

['fuit', 'cerveau', 'urss', 'exod', 'inte¬']

In [0]:
count_mot=c.Counter()
for mot in mots:
  count_mot[mot]+=1
len(count_mot)

25742

In [0]:
#Je prends les 10 mots les plus frequents comme mots clé
mots_cles=count_mot.most_common(12)
mots_cles

[('europ', 14943),
 ('cerveau', 2100),
 ('plus', 1900),
 ('des', 1795),
 ('fuit', 1733),
 ('pay', 1684),
 ('polit', 1375),
 ('européen', 1241),
 ('franc', 1060),
 ('entre', 1051),
 ('comm', 1038),
 ('économ', 1016)]

In [0]:
mots_cles=[mot[0] for mot in mots_cles]
mots_cles.remove('cerveau')
mots_cles.remove('fuit')
mots_cles

['europ',
 'plus',
 'des',
 'pay',
 'polit',
 'européen',
 'franc',
 'entre',
 'comm',
 'économ']

In [0]:
#Creer le dataframe mot_article
t_mot_article=pd.DataFrame(columns=['mot','num_article','frequence'])
j=0
for i in abstract.index:
  for mot in mots_cles:
    text=abstract.loc[i,['abstract']][0].split()
    if mot in text:
      freq=text.count(mot)
      t_mot_article.loc[j,:]=[mot,i+1,freq]
      j+=1
t_mot_article.head()

,mot,num_article,frequence
0,plus,2,1
1,polit,3,1
2,économ,3,3
3,europ,4,1
4,plus,4,1


In [0]:
t_mot_article.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_mot_article.csv',index=False)

#Pays article

In [0]:
#Une fonction qui renvoi la liste des pays du monde
def f_pays():
    list_pays=[]
    for pays in list(py.countries):
        a= re.search("name='"+'.+'+", numeric=",str(pays))
        if a:
            a=a.group(0)[6:-11]
            list_pays.append(a.lower())
    return list_pays

In [0]:
t_pays_article=pd.DataFrame(columns=['nom_pays','num_article'])
list_pays=f_pays()
j=0
for i in abstract.index:
    text=abstract.loc[i,['abstract']][0]
    for pays in list_pays:
      p=re.search(pays,text)
      if p:
        t_pays_article.loc[j,:]=[p.group(0),i+1]
        j+=1
t_pays_article.head()

,nom_pays,num_article
0,réunion,15
1,poland,36
2,oman,43
3,pakistan,62
4,egypt,66


In [0]:
t_pays_article.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_pays_article.csv',index=False)

# Auteur

In [0]:
#Remplacer les Nan par vide
df_auteur=df_auteur.fillna('')
df_auteur.head()

,nom,annee_naissance,annee_decès,bio
0,Kortunov Andrei,,,
1,Bouyssou Rachel,,,
2,Docquier Frédéric,,,Docteur en sciences économiques. - Macroéconom...
3,Rapoport Hillel,1963,,
4,Chauvier Stéphane,1959,,Maître de conférences de philosophie à l'Unive...


In [0]:
#Attribuer un numéro à un auteur
df_auteur['num_auteur']=list(np.arange(1,len(df_auteur)+1))
df_auteur.head()

,nom,annee_naissance,annee_decès,bio,num_auteur
0,Kortunov Andrei,,,,1
1,Bouyssou Rachel,,,,2
2,Docquier Frédéric,,,Docteur en sciences économiques. - Macroéconom...,3
3,Rapoport Hillel,1963,,,4
4,Chauvier Stéphane,1959,,Maître de conférences de philosophie à l'Unive...,5


In [0]:
bios=df_auteur[['num_auteur','bio']]

In [0]:
#Ajouter les professions des auteurs
df_auteur['profession']=''
for i in df_auteur.index:
  bio=df_auteur.loc[i,['bio']][0]
  if bio!='':
    b=bio[:3]
    prof=re.search(r""+b+"\w*(['\(\) ]\w*)*[,\.-]",bio)
    bio=bio.lower()

    if prof:
      df_auteur.loc[i,'profession']=prof.group(0)[:-1]


In [0]:
t_auteur=df_auteur[['num_auteur','nom','annee_naissance','annee_decès','profession']]

In [0]:
t_auteur.head()

,num_auteur,nom,annee_naissance,annee_decès,profession
0,1,Kortunov Andrei,,,
1,2,Bouyssou Rachel,,,
2,3,Docquier Frédéric,,,Docteur en sciences économiques
3,4,Rapoport Hillel,1963,,
4,5,Chauvier Stéphane,1959,,Maître de conférences de philosophie à l'Unive...


In [0]:
t_auteur.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_auteur.csv',index=False)

# Pays auteur


In [0]:
list_pays=f_pays()
bios['nom_pays']=''
for i in bios.index:
  bio=bios.loc[i,['bio']][0]
  if bio!='':
    bio=bio.lower()
    for pays in list_pays:
        pp=re.search(pays,bio)
        if pp:
          bios.loc[i,['nom_pays']]+=' '+pp.group(0)
bios.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,num_auteur,bio,nom_pays
0,1,,
1,2,,
2,3,Docteur en sciences économiques. - Macroéconom...,
3,4,,
4,5,Maître de conférences de philosophie à l'Unive...,


In [0]:
nationalite=bios[['num_auteur','nom_pays']]

In [0]:
t_nationalite=nationalite[nationalite['nom_pays']!='']
t_nationalite.head()

,num_auteur,nom_pays
18,19,france
45,46,france
50,51,france
73,74,france
102,103,france


In [0]:
t_nationalite.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_nationalite.csv',index=False)

# Redige

In [0]:
df_redige

,nom,num_doc,role
0,Kortunov Andrei,1,Auteur
1,Bouyssou Rachel,1,Traducteur
2,Docquier Frédéric,2,Auteur
3,Rapoport Hillel,2,Auteur
4,Chauvier Stéphane,3,Auteur
...,...,...,...
5140,Barseghian Jeanne,4996,Auteur
5141,Guignier Armelle,4997,Auteur
5142,Comte Françoise,4998,Auteur
5143,Coussy Jean,4999,Auteur


In [0]:
df_redige['num_doc']=df_redige['num_doc'].astype(int)

In [0]:
df_redige['num_auteur']=list(np.arange(1,len(df_redige)+1))

In [0]:
t_redige=df_redige[['num_auteur','num_doc']]
t_redige.head()

,num_auteur,num_doc
0,1,1
1,2,1
2,3,2
3,4,2
4,5,3


In [0]:
t_redige.to_csv(sep=',', path_or_buf='/content/drive/My Drive/Colab Notebooks/t_redige.csv',index=False)